In [1]:
from stl import mesh
import numpy as np
import datetime

In [2]:
time_start = datetime.datetime.now()

In [3]:
def create_voxel(side, tx, ty, tz):
    vertices = np.array([
       [tx + 0,     ty + 0,      tz + 0],
       [tx + side,  ty + 0,      tz + 0],
       [tx + side,  ty + side,   tz + 0],
       [tx + 0,     ty + side,   tz + 0],
       [tx + 0,     ty + 0,      tz + side],
       [tx + side,  ty + 0,      tz + side],
       [tx + side,  ty + side,   tz + side],
       [tx + 0,     ty + side,   tz + side]
    ])

    faces = np.array([
        [3, 2, 1],
        [3, 1, 0],
        [0, 1, 5],#
        [5, 4, 0],
        [7, 3, 0],
        [0, 4, 7],#
        [1, 2, 6],#
        [6, 5, 1],
        [2, 3, 6],#
        [3, 7, 6],#
        [4, 5, 6],#
        [6, 7, 4]
     ])
    voxel = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            voxel.vectors[i][j] = vertices[f[j],:]
    return voxel

In [4]:
def voxel_in_hole(holes, x, y):
    comp_error = 0
    for hole in holes:
        if  x + comp_error >= hole['x'][0] and x - comp_error < hole['x'][1] and \
            y + comp_error >= hole['y'][0] and y - comp_error < hole['y'][1]:
            return True
    return False

In [5]:
depth = 3
side = 1/(3**depth)

In [6]:
# holes
holes = []
side = 1
round_factor = 14
for d in range(1, depth+1):
    side /= 3
    for x in range(1,3**d,3):
        for y in range(1,3**d,3):           
            holes.append({
                'x':[round(side*x,round_factor), round(side*x+side,round_factor)],
                'y':[round(side*y,round_factor), round(side*y+side,round_factor)]
            })

In [7]:
sponge = None
for x in range(3**depth):
    for y in range(3**depth):
        for z in range(3**depth):
            rx = round(x/(3**depth),round_factor)
            ry = round(y/(3**depth),round_factor)
            rz = round(z/(3**depth),round_factor)
            if  voxel_in_hole(holes, ry, rz) or \
                voxel_in_hole(holes, rx, ry) or \
                voxel_in_hole(holes, rx, rz):
                continue
            if sponge is None:
                sponge = create_voxel(side, x/(3**depth), y/(3**depth), z/(3**depth))
            else:
                new_voxel = create_voxel(side, x/(3**depth), y/(3**depth), z/(3**depth))
                sponge = mesh.Mesh(np.concatenate([sponge.data, new_voxel.data]))

In [8]:
sponge.save('numpy_stl_example_04.stl')

In [9]:
time_end = datetime.datetime.now()
print('spent', (time_end - time_start).seconds, 'seconds')

spent 24 seconds
